In [1]:
from pathlib import Path
import json
import socket

import pandas as pd
import geopandas as gpd

from pyagnps import soil_data_market as sdm

from sqlalchemy import URL, create_engine, text as sql_text
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import SQLAlchemyError

In [2]:
# DATABASE SETUP
credentials = Path("../../inputs/db_credentials.json")
with open(credentials, "r") as f:
    credentials = json.load(f)

user = credentials["user"]
password = credentials["password"]
host = credentials["host"]
port = credentials["port"]
database = credentials["database"]

url_object = URL.create(
    "postgresql",
    username=user,
    password=password,
    host=host,
    port=port,
    database=database
)

# create a SQLAlchemy engine object
engine = create_engine(url_object)

nodename = socket.gethostname()

In [3]:
raster_CDL_path = Path('D:/AIMS/Datasets/Management/CDL_Annual/CDL_2022.tif')
path_to_management_class_names = Path('../../inputs/management/CDL_All_Codes.csv')

In [4]:
df_cdl = pd.read_csv(path_to_management_class_names)

In [5]:
df_cdl.head()

,Value,Class_Name,Count2021,Src_Year,Value1,RGB_R,RGB_G,RGB_B,Comments
0,0,Background,0,2021.0,0,0.0,0.0,0.0,"Added 0 so the background presents as null, no..."
1,1,Corn,427057895,2021.0,1,255.0,211.0,0.0,NaN
2,2,Cotton,57641094,2021.0,2,255.0,37.0,37.0,NaN
3,3,Rice,12260890,2021.0,3,0.0,168.0,226.0,NaN
4,4,Sorghum,40371933,2021.0,4,255.0,158.0,9.0,NaN


In [6]:
dico = df_cdl[['Value','Class_Name']].set_index('Value').to_dict(orient='dict')['Class_Name']
# dico = df_cdl.set_index('CDL_Value').loc[:,['Mgmt_Schd_ID']].to_dict()
dico = {key: '' if isinstance(value,float) else value for key, value in dico.items()}

In [7]:
classes = set(dico.values())

# Remove classes that need 
already_valid_classes = classes - set(["", "Clouds_No_Data", "Developed"])

reversed_already_valid_dico = {dico[val]: val for val in dico if dico[val] in already_valid_classes}

In [8]:
thuc_id = '1944'

- Loop through T-HUCs
  - Create `CDL_Value_2022` column
  - Populate with matching CDL Value for known categories i.e. anything different than "NaN", "Developed", "Clouds_No_Data" and give -1 to the three mentioned

In [9]:
invalid_field_ids_to_fix = ("'NaN'", "'Clouds_No_Data'", "'Developed'")

with engine.connect() as connection:
    try:
        # Make sure that mgmt_field_id column is indeed text 
        # create a column for cdl value 2022
        # if the mgmt field values are NULL, NaN, Clouds_No_Data, or Developed the cdl_value is set to -1
        query = f"""
        ALTER TABLE thuc_{thuc_id}_annagnps_cell_data_section
        ALTER COLUMN mgmt_field_id SET DATA TYPE text USING CAST(mgmt_field_id AS text);

        ALTER TABLE thuc_{thuc_id}_annagnps_cell_data_section
        ADD COLUMN IF NOT EXISTS cdl_value_2022 INT;

        UPDATE thuc_{thuc_id}_annagnps_cell_data_section 
        SET cdl_value_2022 = CASE 
            WHEN mgmt_field_id IS NULL OR mgmt_field_id IN ({', '.join(invalid_field_ids_to_fix)}) THEN -1 
            ELSE NULL 
        END
        """

        connection.execute(sql_text(query))
        connection.commit()

    except Exception as e:
        print(f"Error for THUC {thuc_id}")
        print(e)
        connection.rollback()

Find mgmt_field_id that have a cdl_value_2022 different than -1 we apply the mapping

In [10]:
# query = 'SELECT * FROM usa_valid_soil_layers_data WHERE "Soil_ID" = 568326'
query = f"SELECT mgmt_field_id FROM thuc_{thuc_id}_annagnps_cell_data_section WHERE cdl_value_2022 <> -1 OR cdl_value_2022 IS NULL"

df = pd.read_sql_query(sql=sql_text(query), con=engine.connect())
df = df.drop_duplicates().reset_index(drop=True) # Find the unique classes present in that THUC

In [13]:
with engine.connect() as connection:
    try:

        query = f"""
        UPDATE thuc_{thuc_id}_annagnps_cell_data_section
        SET cdl_value_2022 = CASE WHEN mgmt_field_id = :Mgmt_Field_ID THEN :CDL_Value_2022 ELSE -1 END
        WHERE mgmt_field_id = :Mgmt_Field_ID
        """
       
        for _, class_name in df.iterrows():
            name = class_name['mgmt_field_id']
            connection.execute(sql_text(query), {"CDL_Value_2022": reversed_already_valid_dico[name], "Mgmt_Field_ID": name})

        connection.commit()

    except Exception as e:
        print(f"Error for THUC {thuc_id}")
        print(e)
        connection.rollback()

Query remaining cells and apply zonal statistics to only those

In [14]:
# Get cell_ids
query = f"SELECT cell_id FROM thuc_{thuc_id}_annagnps_cell_data_section WHERE cdl_value_2022 = -1 OR cdl_value_2022 IS NULL"

df = pd.read_sql_query(sql=sql_text(query), con=engine.connect())

In [16]:
cells_ids_to_reprocess = tuple(df['cell_id'].to_list())

In [17]:
# Collect thuc cells geometry from database

query = f"SELECT * FROM thuc_{thuc_id}_annagnps_cell_ids WHERE dn in {cells_ids_to_reprocess}"

with engine.connect() as conn:
    cells = gpd.read_postgis(sql=sql_text(query), con=conn, geom_col="geom")

utm = cells.estimate_utm_crs()
cells = cells.to_crs(utm)


In [18]:
cells

,fid,dn,geom
0,1,3295452,"POLYGON ((511456.431 3310677.175, 511456.431 3..."
1,2,3295451,"POLYGON ((511516.431 3310617.175, 511516.431 3..."
2,3,3295442,"POLYGON ((510856.431 3310317.175, 510856.431 3..."
3,4,3295453,"POLYGON ((511396.431 3310467.175, 511396.431 3..."
4,5,3295463,"POLYGON ((511066.431 3310107.175, 511066.431 3..."
...,...,...,...
3303959,3303959,910552,"POLYGON ((467956.431 2881707.175, 467956.431 2..."
3303960,3303960,910603,"POLYGON ((467836.431 2882187.175, 467836.431 2..."
3303961,3303961,910551,"POLYGON ((468406.431 2882307.175, 468406.431 2..."
3303962,3303963,910601,"POLYGON ((467836.431 2882067.175, 467836.431 2..."


Doing the plurality analysis using the raster

In [19]:
cells = sdm.assign_attr_zonal_stats_raster_layer(cells, raster_CDL_path, agg_method='majority', attr='CDL_Value')


c:\Users\Luc\projects\pyagnps\.venv\Lib\site-packages\rasterstats\io.py:328: NodataWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn(


In [20]:
cells

,fid,dn,geom,CDL_Value
0,1,3295452,"POLYGON ((511456.431 3310677.175, 511456.431 3...",None
1,2,3295451,"POLYGON ((511516.431 3310617.175, 511516.431 3...",None
2,3,3295442,"POLYGON ((510856.431 3310317.175, 510856.431 3...",None
3,4,3295453,"POLYGON ((511396.431 3310467.175, 511396.431 3...",None
4,5,3295463,"POLYGON ((511066.431 3310107.175, 511066.431 3...",None
...,...,...,...,...
3303959,3303959,910552,"POLYGON ((467956.431 2881707.175, 467956.431 2...",None
3303960,3303960,910603,"POLYGON ((467836.431 2882187.175, 467836.431 2...",None
3303961,3303961,910551,"POLYGON ((468406.431 2882307.175, 468406.431 2...",None
3303962,3303963,910601,"POLYGON ((467836.431 2882067.175, 467836.431 2...",None


In [29]:
cells2 = cells.copy(deep=True)

In [34]:
try:
    cells['CDL_Value'] = cells['CDL_Value'].astype('Int32')
except Exception as e:
    print(e)

Cannot interpret 'Int32Dtype()' as a data type


In [36]:
# cells.loc[cells['CDL_Value']==0, 'CDL_Value'] = 81 # Set 0 value to 81 = Cloud_No_Data
cells['Mgmt_Field_ID'] = cells['CDL_Value'].map(dico)
cells = cells.rename(columns={"dn": "cell_id"})

In [37]:
cells

,fid,cell_id,geom,CDL_Value,Mgmt_Field_ID
0,1,3295452,"POLYGON ((511456.431 3310677.175, 511456.431 3...",None,None
1,2,3295451,"POLYGON ((511516.431 3310617.175, 511516.431 3...",None,None
2,3,3295442,"POLYGON ((510856.431 3310317.175, 510856.431 3...",None,None
3,4,3295453,"POLYGON ((511396.431 3310467.175, 511396.431 3...",None,None
4,5,3295463,"POLYGON ((511066.431 3310107.175, 511066.431 3...",None,None
...,...,...,...,...,...
3303959,3303959,910552,"POLYGON ((467956.431 2881707.175, 467956.431 2...",None,None
3303960,3303960,910603,"POLYGON ((467836.431 2882187.175, 467836.431 2...",None,None
3303961,3303961,910551,"POLYGON ((468406.431 2882307.175, 468406.431 2...",None,None
3303962,3303963,910601,"POLYGON ((467836.431 2882067.175, 467836.431 2...",None,None


In [38]:
cells.explore(column='Mgmt_Field_ID', categorical=True)

AxisError: axis 1 is out of bounds for array of dimension 0

In [39]:
data_to_update = cells[["cell_id", "Mgmt_Field_ID", "CDL_Value"]].to_dict(orient="records")

Populate Data

In [40]:
try:
    # create a session factory
    Session = sessionmaker(bind=engine)
    # create a new session
    session = Session()
    # create a transaction
    transaction = session.begin()

    # execute your update query here
    query = f"""UPDATE thuc_{thuc_id}_annagnps_cell_data_section 
    SET mgmt_field_id = :Mgmt_Field_ID,
        cdl_value_2022 = :CDL_Value
    WHERE cell_id = :cell_id"""
    
    session.execute(sql_text(query), data_to_update)
    # commit the transaction
    transaction.commit()

except Exception as e:
    print(e)
    # rollback the transaction on error
    transaction.rollback()

finally:
    # close the session
    session.close()

Check that it worked

In [41]:
# query = 'SELECT * FROM usa_valid_soil_layers_data WHERE "Soil_ID" = 568326'
query = f"SELECT cell_id, mgmt_field_id, cdl_value_2022 FROM thuc_{thuc_id}_annagnps_cell_data_section ORDER BY cell_id"

df = pd.read_sql_query(sql=sql_text(query), con=engine.connect())
df.head()

,cell_id,mgmt_field_id,cdl_value_2022
0,22,None,None
1,23,None,None
2,31,None,None
3,32,None,None
4,33,None,None


In [42]:
df[df['mgmt_field_id'].isnull()].empty

False